In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
np.random.seed(42)

In [2]:
training_data = np.loadtxt("train.csv", delimiter=",")
testing_data = np.loadtxt("test.csv", delimiter=",")

# Split training data
X = training_data[:, 1:]
y = training_data[:, 0]
y[y == 0] = -1 # Convert labels from {0,1} to {-1,1}
X_train = X[:4000] # Use the first 4000 samples as training data
y_train = y[:4000]
X_val = X[4000:] # Remaining samples as validation data
y_val = y[4000:]

# Split testing data
X_test = testing_data[:, 1:]
y_test = testing_data[:, 0]
y_test[y_test == 0] = -1

In [3]:
# Normalize the data
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

## Question 2

In [4]:
# Function to return the trained SVM model parameters using primal form
def svm_train_primal(data_train, label_train, regularisation_para_C):
    num_samples, num_features = data_train.shape

    w = cp.Variable(num_features, value=np.zeros(num_features)) # Weight vector
    b = cp.Variable(value=0.0) # Bias term
    xi = cp.Variable(num_samples) # Slack variables

    # Define objective function
    objective = cp.Minimize(0.5 * cp.norm(w,2)**2 + regularisation_para_C * cp.sum(xi) / num_samples)

    # Define constraints
    constraints = [label_train[i] * (data_train[i] @ w + b) >= 1 - xi[i] for i in range(num_samples)]
    constraints += [xi >= 0]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # Return SVM model parameters
    svm_primal_model = {"w": w.value, "b": b.value, "xi": xi.value}
    return svm_primal_model

In [5]:
# Function to predict on testing data and calculate accuracy
def svm_predict_primal(data_test, label_test, svm_primal_model):
    w = svm_primal_model["w"]
    b = svm_primal_model["b"]

    pred = np.sign(data_test @ w + b)
    accuracy = np.mean(pred == label_test)

    return accuracy

In [6]:
# Train primal model with C=100 and then test
svm_primal_model = svm_train_primal(X_train, y_train, 100)
val_accuracy = svm_predict_primal(X_val, y_val, svm_primal_model)
test_accuracy = svm_predict_primal(X_test, y_test, svm_primal_model)

print(f"Bias (b) of SVM model is: {svm_primal_model['b']}")
print(f"Sum of all dimensions of weight vector (w) is: {np.sum(svm_primal_model['w'])}")
print(f"Resulting accuracy on validation data is: {val_accuracy}")
print(f"Resulting accuracy on test data is: {test_accuracy}")

Bias (b) of SVM model is: 1.7798137266344365
Sum of all dimensions of weight vector (w) is: -0.14521544474819548
Resulting accuracy on validation data is: 0.9695555555555555
Resulting accuracy on test data is: 0.968


## Question 3

In [7]:
# Function to return the trained SVM model parameters using dual form
def svm_train_dual(data_train, label_train, regularisation_para_C):
    num_samples, num_features = data_train.shape

    alpha = cp.Variable(num_samples) # Optimization variable
    # kernel = data_train @ data_train.T

    # Define objective function
    objective = cp.Maximize(cp.sum(alpha) - 0.5 * cp.sum_squares(cp.multiply(alpha, label_train) @ data_train))
    
    # Define constraints
    constraints = [
        alpha >= 0,
        alpha <= regularisation_para_C / num_samples,
        cp.sum(cp.multiply(alpha, label_train)) == 0
    ]
    
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.CVXOPT)

    # Return SVM model parameter
    svm_dual_model = {"alpha": alpha.value}
    return svm_dual_model

In [8]:
# Train dual model with C=100
svm_dual_model = svm_train_dual(X_train, y_train, 100)

print(f"Sum of all dimensions of optimal alpha is: {np.sum(svm_dual_model['alpha'])}")

Sum of all dimensions of optimal alpha is: 7.281635912065749


## Question 4

In [9]:
# To obtain primal problem solution from dual problem
alpha = svm_dual_model['alpha']
num_samples, num_features = X_train.shape

# Calculate weight vector
w_star = np.sum(alpha[:, None] * y_train[:, None] * X_train, axis=0)

# Calculate bias term by identifying support vectors where their alpha > 0
support_vectors = np.where(alpha > 1e-5)[0]
bias_support_vectors = y_train[support_vectors] - np.dot(X_train[support_vectors], w_star)
b_star = np.mean(bias_support_vectors) # Average bias values

print(f"Sum of all dimensions of weight vector for primal problem from dual solution is: {np.sum(w_star)}")
print(f"Solution of bias term from primal problem from dual solution is: {b_star}")

Sum of all dimensions of weight vector for primal problem from dual solution is: -0.145205601444225
Solution of bias term from primal problem from dual solution is: 1.7389405965797873


## Question 5

In [10]:
# To find support vectors from primal solutions
constraint = X_train@svm_primal_model["w"] + svm_primal_model["b"] # Constraint of primal problem

# Identify support vectors that fit the constraint
support_vectors = (np.multiply(y_train, constraint) - 1 + svm_primal_model["xi"]) <= 1e-4
primal_support_vectors = X_train[support_vectors]

print(f"Number of support vectors found from primal solutions is: {len(primal_support_vectors)}")

Number of support vectors found from primal solutions is: 392


## Question 6

In [11]:
# To find support vectors from dual solution
support_vectors = np.where(alpha > 1e-5)[0] # Identify support vectors that fit the dual constraint
dual_support_vectors = X_train[support_vectors]

print(f"Number of support vectors found from dual solution is: {len(dual_support_vectors)}")

Number of support vectors found from dual solution is: 392


## Question 7

In [12]:
# Function to find optimal regularisation parameter C using grid search
def find_optimal_C(data_train, label_train, data_val, label_val, C_values):
    optimal_C = None
    optimal_accuracy = 0

    # Train and test svm model with different values of C using primal solution
    for regularisation_para_C in C_values:
        svm_primal_model = svm_train_primal(data_train, label_train, regularisation_para_C)
        val_accuracy = svm_predict_primal(data_val, label_val, svm_primal_model)

        # If current validation accuracy is higher than current optimal accuracy, update
        if val_accuracy > optimal_accuracy:
            optimal_C = regularisation_para_C
            optimal_accuracy = val_accuracy
    
    return optimal_C, optimal_accuracy

In [13]:
# To find optimal C
C_values = [2**-10, 2**-8, 2**-6, 2**-4, 2**-2, 2**0, 2**2, 2**4, 2**8, 2**10] # 2**-10, 2**100, optimal - 0.0039...
optimal_C, optimal_accuracy = find_optimal_C(X_train, y_train, X_val, y_val, C_values)

print(f"Optimal regularisation parameter C is: {optimal_C} with validation accuracy of {optimal_accuracy}")

# To report test accuracy using optimal C
svm_primal_model = svm_train_primal(X_train, y_train, optimal_C)
test_accuracy = svm_predict_primal(X_test, y_test, svm_primal_model)

print(f"Test accuracy of primal model using optimal C is: {test_accuracy}")

Optimal regularisation parameter C is: 4 with validation accuracy of 0.9748888888888889
Test accuracy of primal model using optimal C is: 0.9746666666666667


## Question 8

In [14]:
# To perform classification with linear SVM using Scikit-Learn SVM
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Testing with penalty "l1" and loss "squared_hinge"
model = LinearSVC(penalty='l1', loss='squared_hinge', C=optimal_C, max_iter=10000, random_state=42)
model.fit(X_train, y_train)

# Predict on validation set
val_predict = model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predict)

# Predict on test set
test_predict = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predict)

print(f"LinearSVM model with penalty 'l1' and loss 'squared_hinge' with optimal C '{optimal_C}':")
print(f"Validation accuracy: {val_accuracy} and test accuracy: {test_accuracy}")

LinearSVM model with penalty 'l1' and loss 'squared_hinge' with optimal C '4':
Validation accuracy: 0.964 and test accuracy: 0.9713333333333334


In [15]:
# Testing with penalty "l2" and loss "squared_hinge"
model = LinearSVC(penalty='l2', loss='squared_hinge', C=optimal_C, max_iter=10000, random_state=42)
model.fit(X_train, y_train)

# Predict on validation set
val_predict = model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predict)

# Predict on test set
test_predict = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predict)

print(f"LinearSVM model with penalty 'l2' and loss 'squared_hinge' with optimal C '{optimal_C}':")
print(f"Validation accuracy: {val_accuracy} and test accuracy: {test_accuracy}")

LinearSVM model with penalty 'l2' and loss 'squared_hinge' with optimal C '4':
Validation accuracy: 0.964 and test accuracy: 0.9713333333333334


*Hence, final test accuracy of classification with LinearSVM model using Scikit-learn is 0.9713 or 97.13%*